In [ ]:
import imutils
import itertools
import cv2
import colorsys
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 8]

In [ ]:
vid = cv2.VideoCapture('/var/tmp/cam1.mp4')
def make_frames():
    i = 0
    while True:
        status, frame = vid.read()
        if not status:
            break
        yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
def show(*args):
    if len(args) > 1:
        imgs = np.hstack(tuple(args))
    else:
        imgs = args[0]
    plt.imshow(imgs)
    plt.show()
frames = list(itertools.islice(make_frames(), 320, 820))

In [ ]:
len(frames)

In [ ]:
show(*frames[250:290:10])

In [ ]:
s=50
gamma = 0.9
base = cv2.absdiff(frames[s], frames[s+1])
for i in range(s+1, 350):
    if i % 20 == 0:
        cv2.multiply(base, (gamma, gamma, gamma, gamma), base)
    cv2.add(base, cv2.absdiff(cv2.GaussianBlur(frames[i+1], (21, 21), 0), cv2.GaussianBlur(frames[i], (21, 21), 0)), base)
show(base)

In [ ]:
s=220
gamma = 0.9
base = cv2.absdiff(frames[s], frames[s+1])
for i in range(s+1, s+20):
    cv2.add(base, cv2.absdiff(cv2.GaussianBlur(frames[i+1], (37, 37), 0), cv2.GaussianBlur(frames[i], (37, 37), 0)), base)
mask = cv2.cvtColor(base, cv2.COLOR_RGB2GRAY).astype(np.float32) / 256
cv2.multiply(mask, 2, mask)
mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
source = frames[s+10].copy().astype(np.float32) / 256
cut = cv2.multiply(source, mask_rgb)
inv_mask = cv2.cvtColor(1.0 - mask, cv2.COLOR_GRAY2RGB)
back = cv2.multiply(frames[150].astype(np.float32)/256, inv_mask)
show(frames[s+10].astype(np.float32)/256, cv2.add(cut, back))

In [ ]:
def add_to(dest, block):
    base = cv2.absdiff(block[0], block[1])
    for i in range(1, len(block) - 1):
        cv2.add(base, cv2.absdiff(cv2.GaussianBlur(block[i+1], (37, 37), 0), cv2.GaussianBlur(block[i], (37, 37), 0)), base)
    mask = cv2.cvtColor(base, cv2.COLOR_RGB2GRAY).astype(np.float32) / 256
    cv2.multiply(mask, 2, mask)
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    source = block[len(block)//2].copy().astype(np.float32) / 256
    cut = cv2.multiply(source, mask_rgb)
    inv_mask = cv2.cvtColor(1.0 - mask, cv2.COLOR_GRAY2RGB)
    # show(mask_rgb, inv_mask)
    
    back = cv2.multiply(dest, inv_mask)
    cv2.add(back, cut, dest)

def frames_from(n, w):
    return frames[max(n, 0) : min(n+w, len(frames))]

def train(s=80, w=20):
    d = frames[0].copy().astype(np.float32) / 256
    for i in range(0, len(frames), s):
        add_to(d, frames_from(i, w))
    return d

show(train(80, 2), train(80, 20), train(80, 40))

In [ ]:
show(train(40), train(120), train(180))

In [ ]:
show(train(10, 5))